# Human Activity Recognition using Inertial sensors and Neural Networks

Elia Bonetto, Filippo Rigotto. 

Deptartment of Information Engineering, University of Padova, Italy.

Human Data Analytics, a.y. 2018/2019

In [0]:
!nvidia-smi

In [0]:
from IPython.display import Image, clear_output

In [0]:
import os
from google.colab import drive
drive.mount('/content/drive/')
clear_output()
!ls /content/drive/My\ Drive/hda-project
os.chdir("/content/drive/My Drive/hda-project")

In [0]:
import json
import logging
from datetime import datetime
from pprint import pprint

import numpy as np
import scipy as sp
import scipy.io
from sklearn.model_selection import train_test_split

import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline
mpl.rcParams['figure.figsize'] = (16,10)
mpl.rcParams['axes.grid'] = True

import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.layers import Input, Dense, Activation, BatchNormalization, Flatten, Conv1D, MaxPooling1D, Dropout
#from tensorflow.keras.layers import Conv2D, ZeroPadding2D, AveragePooling2D, MaxPooling2D, Dropout, GlobalMaxPooling2D, GlobalAveragePooling2D
from tensorflow.keras.models import Model
#logging.getLogger('tensorflow').disabled = True

from dataset import ars_ds

## Data preprocessing

### Load dataset

Note that `num_classes = 16` instead of 17 because there are no elements assigned to the last 'TRANSIT' class.

In [0]:
ds = ars_ds.ARSDataset('dataset/ARS_DLR.npz')
imu,labels = ds.get()

num = imu.shape[0]
num_classes = np.unique(labels).size
print("Num classes: " + str(num_classes))

Temporary reassign labels (until definitive): group walking, jumping and transitions.

DO NOT TRUST `get_label_encode/decode_map()` after this passage.


In [0]:
dmap = ds.get_label_decode_map()
newmap = {
    0:0, 1:1, 2:2, 3:3, 4:4, 5:5, 6:6, # unaltered
    7:1,   8:1, # = walking
    9:2,  10:2, 11:2, # = jumping
    12:7, 13:7, 14:7, 15:7, 16:7 # = transitions
}
pprint(dmap)
print(newmap)

labels = np.array([newmap[i] for i in labels])
num_classes = np.unique(labels).size
print("Num classes: " + str(num_classes))

### Train-test split

`random_state` is the seed of the PRNG.

In [0]:
X_train, X_test, Y_train, Y_test = train_test_split(imu, labels, test_size=0.2, random_state=1)

print("IMU shape:     " + str(imu.shape))
print("Labels shape:  " + str(labels.shape))
print("X_train shape: " + str(X_train.shape))
print("Y_train shape: " + str(Y_train.shape))
print("X_test shape:  " + str(X_test.shape))
print("Y_test shape:  " + str(Y_test.shape))

In [0]:
def create_datasets(X_train, X_test, Y_train, Y_test, batch_size):
    train_dataset = tf.data.Dataset.from_tensor_slices((X_train, Y_train))
    train_dataset = train_dataset.shuffle(len(X_train))
    #train_dataset = train_dataset.repeat()
    train_dataset = train_dataset.map(lambda data, label: (tf.expand_dims(data, 1), label))
    train_dataset = train_dataset.batch(batch_size=batch_size)
    train_dataset = train_dataset.prefetch(buffer_size=1)

    test_dataset = tf.data.Dataset.from_tensor_slices((X_test, Y_test))
    #test_dataset = test_dataset.repeat()
    test_dataset = test_dataset.map(lambda data, label: (tf.expand_dims(data, 1), label))
    test_dataset = test_dataset.batch(batch_size=batch_size)
    test_dataset = test_dataset.prefetch(buffer_size=1)
    return train_dataset, test_dataset

## Models

In [0]:
def conv1model(input_shape, num_classes):
    # Define the input placeholder as a tensor with shape input_shape. Think of this as your input image!
    X_input = Input(shape=input_shape)

    # Zero-Padding: pads the border of X_input with zeroes
    # X = ZeroPadding2D((3, 3))(X_input)

    # CONV -> Batch Normalization -> ReLU Block applied to X
    X = Conv1D(32, 5, name='conv0')(X_input)
    X = BatchNormalization(axis=1, name='bn0')(X)
    X = Activation('relu')(X)

    # MAXPOOL
    X = MaxPooling1D(2, name='max_pool0')(X)

    # FLATTEN X (means convert it to a vector) + FULLYCONNECTED
    X = Flatten()(X)
    #X = Dense(128, activation='relu', name='fc')(X)
    X = Dense(num_classes, activation='softmax', name='softmax')(X)

    model = Model(inputs = X_input, outputs = X, name='Conv1Model')
    return model

## Training and evaluation

In [0]:
def run_model(model, train_dataset, test_dataset, config):
    """Generic method to build a model, train and evaluate performances."""

    out_folder = os.path.join('output', datetime.now().strftime('%Y%m%d-%H%M%S')+'_'+model.name)
    if not os.path.exists(out_folder):
        os.mkdir(out_folder)
    
    # print and save model summary
    model.summary()
    with open(os.path.join(out_folder, 'summary.txt'),'w') as sfile:
        model.summary(print_fn=lambda x: sfile.write(x+'\n'))

    # use and save config
    with open(os.path.join(out_folder, 'config.json'),'w') as cfile:
        json.dump(config, cfile, indent=2)

    # compile model
    model.compile(optimizer=config['optimizer'], loss=config['loss'], metrics=['accuracy'])

    # train model, save final state and history
    history = model.fit(train_dataset, epochs=config['epochs'], validation_data=test_dataset)
    model.save(os.path.join(out_folder, 'model.h5'))
    with open(os.path.join(out_folder, 'history.json'),'w') as hfile:
        json.dump(history.history, hfile, indent=2)

    # evaluate model, save results
    preds = model.evaluate(test_dataset)
    print ("Loss = " + str(preds[0]))
    print ("Test Accuracy = " + str(preds[1]) + " = " + str(preds[1]*100))
    with open(os.path.join(out_folder, 'evaluation.json'),'w') as efile:
        json.dump({'loss':preds[0], 'accuracy':preds[1]}, efile, indent=2)

    # plot and save loss and accuracy
    plt.figure()
    plt.plot(history.history['loss'], label='Train loss')
    plt.plot(history.history['val_loss'], label='Val loss')
    plt.legend()
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.tight_layout()
    fname = os.path.join(out_folder, 'plot-loss')
    plt.savefig(fname+'.png')
    plt.savefig(fname+'.pdf', format='pdf')

    plt.figure()
    plt.plot(history.history['acc'], label='Train loss')
    plt.plot(history.history['val_acc'], label='Val loss')
    plt.legend()
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.tight_layout()
    fname = os.path.join(out_folder, 'plot-accuracy')
    plt.savefig(fname+'.png')
    plt.savefig(fname+'.pdf', format='pdf')

Define here models and config files to test

In [0]:
batch_size = 32
train_dataset, test_dataset = create_datasets(X_train, X_test, Y_train, Y_test, batch_size)

input_shape = X_train.shape

In [0]:
config = {
    'optimizer': 'adam',
    'loss': 'categorical_crossentropy',
    'epochs': 1,
    'batch_size': batch_size, # inferred from dataset
    'batch_per_epoch': int(np.ceil(len(X_train)/batch_size)) # use only with repeat() in dataset
}
model = conv1model(input_shape, num_classes)
run_model(model, train_dataset, test_dataset, config)